<a href="https://colab.research.google.com/github/tanvisharma0109/ML-with-Python/blob/main/Assignment2%263.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

path_2_train_file = '/content/drive/MyDrive/assignment_new_2020/assignment_new_2020/train.csv'
df = pd.read_csv(path_2_train_file)

In [ ]:
df.shape

(1999999, 332)

In [ ]:
colnames=df.columns

In [ ]:
import pandas
import random

n = 1999999 #number of records in file
s = 40000 #desired sample size
filename = path_2_train_file
skip = sorted(random.sample(range(n),n-s))
df_sample = pandas.read_csv(filename, skiprows=skip)

In [ ]:

df_sample.shape

(40000, 332)

In [ ]:
df_sample.columns=colnames

In [ ]:
df_sample.head()

,day,user_id,item_id,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,...,col291,col292,col293,col294,col295,col296,col297,col298,col299,col300,col301,col302,col303,col304,col305,col306,col307,col308,col309,col310,col311,col312,col313,col314,col315,col316,col317,col318,col319,col320,col321,col322,col323,col324,col325,col326,col327,col328,col329,label
0,466,278613,7523547,0.0,0.4904,0.2309,0.5720,0.0,0.1830,0.0,0.0,0.1154,0.0,0.0,0.000,0.7123,0.0,0.0000,0.0,0.0000,0.000,0.183,0.0000,0.4510,0.000,0.3463,0.0000,0.0000,0.1154,0.3835,0.2681,0.000,0.0000,0.0000,0.0000,0.4904,0.4618,0.0,0.7687,0.0000,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.3739,0.0,0.0,0.0,0.0,0.6292,0.3221,0.0,0.0,0.0,0.1610,0.0,0.0,0.3221,0.2552,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,466,701596,7523515,0.0,0.3116,0.0000,0.4643,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.000,0.6381,0.0,0.1342,0.0,0.0000,0.000,0.000,0.4966,0.3768,0.000,0.1342,0.0000,0.0000,0.0000,0.1342,0.2127,0.000,0.0000,0.0000,0.0000,0.4026,0.2127,0.0,0.7574,0.0000,...,0.1667,0.1667,0.0,0.0,0.0,0.0,0.3870,0.0,0.0,0.0,0.0,0.4679,0.2642,0.0,0.0,0.0,0.1667,0.0,0.0,0.1667,0.2642,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,466,7635508,7538264,0.0,0.3066,0.0000,0.6131,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.000,0.6131,0.0,0.0000,0.0,0.0000,0.000,0.000,0.1934,0.0000,0.000,0.3066,0.0000,0.0000,0.0000,0.3066,0.1934,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.6131,0.0000,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.3739,0.0,0.0,0.0,0.0,0.6292,0.3221,0.0,0.0,0.0,0.1610,0.0,0.0,0.3221,0.2552,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,466,246396,7538280,0.0,0.2920,0.3172,0.6012,0.0,0.0000,0.0,0.0,0.2259,0.0,0.0,0.113,0.7908,0.0,0.1130,0.0,0.2623,0.113,0.000,0.3581,0.3908,0.113,0.4519,0.0000,0.3753,0.0000,0.2920,0.2623,0.113,0.1791,0.1130,0.2259,0.6012,0.1791,0.0,0.6130,0.0000,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.3739,0.0,0.0,0.0,0.0,0.6292,0.3221,0.0,0.0,0.0,0.1610,0.0,0.0,0.3221,0.2552,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,466,1093432,7523547,0.0,0.2840,0.3434,0.4527,0.0,0.1223,0.0,0.0,0.1223,0.0,0.0,0.000,0.7711,0.0,0.0000,0.0,0.1223,0.000,0.000,0.5101,0.4527,0.000,0.5196,0.1223,0.0000,0.0000,0.3878,0.1223,0.000,0.2447,0.1939,0.1223,0.4893,0.1223,0.0,0.6678,0.1223,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.3739,0.0,0.0,0.0,0.0,0.6292,0.3221,0.0,0.0,0.0,0.1610,0.0,0.0,0.3221,0.2552,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
del df

In [ ]:
df2 = df_sample.drop(['day','item_id','user_id'], axis=1)

In [ ]:
X = df2.loc[:, df2.columns != "label"]

In [ ]:
y = df2.loc[:, df2.columns == "label"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state = 0, stratify = y)


In [ ]:

from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy = 'minority', random_state = 123)
X_train, y_train = sm.fit_resample(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

In [ ]:

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=123, n_estimators=60)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict_proba(X_test)
y_pred_rfc = rfc.predict(X_test)


In [ ]:

print(rfc.score(X_train, y_train))
print(rfc.score(X_test, y_test))



0.9999763145428707
0.7775833333333333


In [ ]:

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rfc))


              precision    recall  f1-score   support

           0       0.80      0.94      0.86      9047
           1       0.61      0.27      0.38      2953

    accuracy                           0.78     12000
   macro avg       0.70      0.61      0.62     12000
weighted avg       0.75      0.78      0.74     12000



In [ ]:

from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred_rfc))

[[8528  519]
 [2150  803]]


In [ ]:

from sklearn.model_selection import cross_val_score
from numpy import mean
scores = cross_val_score(rfc, X, y, scoring='roc_auc',n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.779


In [ ]:

df_test = pd.read_csv('/content/drive/MyDrive/assignment_new_2020/assignment_new_2020/test.csv')

df_test.head()


,day,user_id,item_id,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,...,col291,col292,col293,col294,col295,col296,col297,col298,col299,col300,col301,col302,col303,col304,col305,col306,col307,col308,col309,col310,col311,col312,col313,col314,col315,col316,col317,col318,col319,col320,col321,col322,col323,col324,col325,col326,col327,col328,col329,label
0,526,5282,8228074,0.0,0.1378,0.1378,0.6052,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.1378,0.4134,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.2756,0.0,0.2756,0.0,0.0000,0.0,0.0000,0.1378,0.0,0.0000,0.0000,0.0000,0.1378,0.3562,0.0,0.6052,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5458,0.0,0.0,0.1722,0.0,0.4451,0.2729,0.0,0.0,0.0,0.0,0.1722,0.0,0.1722,0.1722,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,526,337986,8228074,0.0,0.3173,0.0000,0.2514,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.5488,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.1586,0.0000,0.0,0.0000,0.0000,0.0000,0.3173,0.0000,0.0,0.5270,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5458,0.0,0.0,0.1722,0.0,0.4451,0.2729,0.0,0.0,0.0,0.0,0.1722,0.0,0.1722,0.1722,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,526,3201104,8228074,0.0,0.1843,0.1843,0.3686,0.0,0.0,0.0,0.0,0.1843,0.0000,0.0,0.0000,0.4279,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.1843,0.0,0.0000,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.1843,0.0000,0.0,0.8558,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5458,0.0,0.0,0.1722,0.0,0.4451,0.2729,0.0,0.0,0.0,0.0,0.1722,0.0,0.1722,0.1722,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,526,6331063,8228074,0.0,0.4333,0.1171,0.7129,0.0,0.0,0.0,0.0,0.0000,0.1171,0.0,0.0000,0.6007,0.0,0.0,0.0,0.0,0.0,0.0,0.2342,0.0000,0.0,0.3513,0.0,0.1171,0.0,0.0000,0.2342,0.0,0.1856,0.1171,0.1171,0.4198,0.1171,0.0,0.6889,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5458,0.0,0.0,0.1722,0.0,0.4451,0.2729,0.0,0.0,0.0,0.0,0.1722,0.0,0.1722,0.1722,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,526,652231,8228074,0.0,0.2973,0.0000,0.7067,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.4712,0.0,0.0,0.0,0.0,0.0,0.0,0.2973,0.0000,0.0,0.1486,0.0,0.0000,0.0,0.1486,0.0000,0.0,0.0000,0.0000,0.0000,0.2973,0.0000,0.0,0.6902,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5458,0.0,0.0,0.1722,0.0,0.4451,0.2729,0.0,0.0,0.0,0.0,0.1722,0.0,0.1722,0.1722,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [ ]:

del df_test['label']


In [ ]:

df_test = df_test.drop(['day','item_id','user_id'], axis=1)


In [ ]:

prediction = pd.DataFrame(rfc.predict_proba(df_test), columns=['Prob_0','Prob_1'])
df_test['Prob_0'] = prediction['Prob_0']
df_test['Prob_1'] = prediction['Prob_1']


In [ ]:

Result = pd.DataFrame()


In [ ]:

Result['Prob_0'] = df_test['Prob_0']

Result['Prob_1'] = df_test['Prob_1']

Result.to_csv("Prediction_Results.csv")